In [12]:
import time
import hmac
import requests
import configparser

config = configparser.ConfigParser() 
config.read("C:/Users/tomas/Dropbox/Prop-Trading Business/ftx.cfg") 
ftx_api_url = "https://ftx.com/api//markets/BTC/USD" 
# api and secret
apiKey = config["ftx"]['ftx_api_url']           
secret = config["ftx"]['api_secret']     

ts = int(time.time() * 1000)

request = Request('GET', ftx_api_url)
prepared = request.prepare()
signature_payload = f'{ts}{prepared.method}{prepared.path_url}'.encode()
signature = hmac.new(secret.encode(), signature_payload, 'sha256').hexdigest()

request.headers['FTX-KEY'] = apiKey
request.headers['FTX-SIGN'] = signature
request.headers['FTX-TS'] = str(ts)


prepared

r_session = requests.Session()
response = r_session.send(prepared)
print(response.json)

<bound method Response.json of <Response [200]>>


In [14]:
import time
import hmac
import requests
import configparser
config = configparser.ConfigParser() 
config.read("C:/Users/tomas/Dropbox/Prop-Trading Business/ftx.cfg") 


class FTX_API():
    """
    Initialize FTX api connection 
    """
    def __init__(self):
        self.ts                 = int(time.time() * 1000)
        self.signature_payload  = f'{ts}{prepared.method}{prepared.path_url}'.encode()
        self.signature          = hmac.new(secret.encode(), signature_payload, 'sha256').hexdigest()
        self.session            =  requests.Session() #Estabilish HTTP Persistent Connection - Keep-Alive
        self.api_key            =  config["ftx"]['ftx_api_url'] # Enter Your api key
        self.secret             =  config["ftx"]['api_secret']  # Enter api url
        self.headers            = {'FTX-KEY': self.api_key,
                                    'FTX-SIGN': self.signature,
                                    'FTX-TS': str(self.ts)} # authetication header for secure api requests
#if __name__ == "__main__":


SyntaxError: unexpected EOF while parsing (Temp/ipykernel_30920/3083321868.py, line 23)

In [8]:
import ccxt
import json

# api and secret
apiKey = config["ftx"]['ftx_api_url']           
secret = config["ftx"]['api_secret']         
subaccount = ''                                    

# Exchange Detail
exchange = ccxt.ftx({
    'apiKey' : apiKey ,'secret' : secret ,'enableRateLimit': True
})

# Sub Account Check

if subaccount == "0":
  print("This is Main Account")
else:
  exchange.headers = {
   'FTX-SUBACCOUNT': subaccount,
  }

In [9]:
# Global Variable Setting
pair = 'BTC-PERP'
tf = '5m'

In [10]:
def priceHistdata():
  
  try:
    priceData = pd.DataFrame(exchange.fetch_ohlcv(pair,tf))
  except ccxt.NetworkError as e:
    print(exchange.id, 'fetch_ohlcv failed due to a network error:', str(e))
    priceData = pd.DataFrame(exchange.fetch_ohlcv(pair,tf))
  except ccxt.ExchangeError as e:  
    print(exchange.id, 'fetch_ohlcv failed due to exchange error:', str(e))
    priceData = pd.DataFrame(exchange.fetch_ohlcv(pair,tf))
  except Exception as e:
    print(exchange.id, 'fetch_ohlcv failed with:', str(e))
    priceData = pd.DataFrame(exchange.fetch_ohlcv(pair,tf))

  return priceData

def getPrice():

  try:
    r1 = json.dumps(exchange.fetch_ticker(pair))
    dataPrice = json.loads(r1)
    #print(exchange)
    #print(pair + '=',dataPrice['last'])
  except ccxt.NetworkError as e:
    r1 = json.dumps(exchange.fetch_ticker(pair))
    dataPrice = json.loads(r1)
  except ccxt.ExchangeError as e:
    r1 = json.dumps(exchange.fetch_ticker(pair))
    dataPrice = json.loads(r1)
  except Exception as e:
    r1 = json.dumps(exchange.fetch_ticker(pair))
    dataPrice = json.loads(r1)
  
  return (dataPrice['last'])

In [11]:
data = priceHistdata()

In [12]:
data

,0,1,2,3,4,5
0,1635490500000,61380.0,61447.0,61364.0,61398.0,6.513470e+06
1,1635490800000,61398.0,61398.0,61131.0,61142.0,1.677365e+07
2,1635491100000,61142.0,61238.0,61142.0,61187.0,1.522537e+07
3,1635491400000,61187.0,61207.0,60759.0,60833.0,3.420351e+07
4,1635491700000,60833.0,60928.0,60716.0,60775.0,2.914930e+07
...,...,...,...,...,...,...
1496,1635939300000,63129.0,63269.0,63128.0,63259.0,8.087909e+06
1497,1635939600000,63259.0,63259.0,63209.0,63212.0,3.219758e+06
1498,1635939900000,63212.0,63254.0,63178.0,63206.0,4.492632e+06
1499,1635940200000,63206.0,63247.0,63203.0,63225.0,4.346553e+06


In [15]:
import time
import urllib.parse
from typing import Optional, Dict, Any, List

from requests import Request, Session, Response
import hmac
from ciso8601 import parse_datetime



class FtxClient:
    _ENDPOINT = 'https://ftx.com/api/'

    def __init__(self, api_key=None, api_secret=None, subaccount_name=None) -> None:
        self._session = Session()
        self._api_key = api_key
        self._api_secret = api_secret
        self._subaccount_name = subaccount_name

    def _get(self, path: str, params: Optional[Dict[str, Any]] = None) -> Any:
        return self._request('GET', path, params=params)

    def _post(self, path: str, params: Optional[Dict[str, Any]] = None) -> Any:
        return self._request('POST', path, json=params)

    def _delete(self, path: str, params: Optional[Dict[str, Any]] = None) -> Any:
        return self._request('DELETE', path, json=params)

    def _request(self, method: str, path: str, **kwargs) -> Any:
        request = Request(method, self._ENDPOINT + path, **kwargs)
        self._sign_request(request)
        response = self._session.send(request.prepare())
        return self._process_response(response)

    def _sign_request(self, request: Request) -> None:
        ts = int(time.time() * 1000)
        prepared = request.prepare()
        signature_payload = f'{ts}{prepared.method}{prepared.path_url}'.encode()
        if prepared.body:
            signature_payload += prepared.body
        signature = hmac.new(self._api_secret.encode(), signature_payload, 'sha256').hexdigest()
        request.headers['FTX-KEY'] = self._api_key
        request.headers['FTX-SIGN'] = signature
        request.headers['FTX-TS'] = str(ts)
        if self._subaccount_name:
            request.headers['FTX-SUBACCOUNT'] = urllib.parse.quote(self._subaccount_name)

    def _process_response(self, response: Response) -> Any:
        try:
            data = response.json()
        except ValueError:
            response.raise_for_status()
            raise
        else:
            if not data['success']:
                raise Exception(data['error'])
            return data['result']

    def list_futures(self) -> List[dict]:
        return self._get('futures')

    def list_markets(self) -> List[dict]:
        return self._get('markets')

    def get_orderbook(self, market: str, depth: int = None) -> dict:
        return self._get(f'markets/{market}/orderbook', {'depth': depth})

    def get_trades(self, market: str) -> dict:
        return self._get(f'markets/{market}/trades')

    def get_account_info(self) -> dict:
        return self._get(f'account')

    def get_open_orders(self, market: str = None) -> List[dict]:
        return self._get(f'orders', {'market': market})
    
    def get_order_history(self, market: str = None, side: str = None, order_type: str = None, start_time: float = None, end_time: float = None) -> List[dict]:
        return self._get(f'orders/history', {'market': market, 'side': side, 'orderType': order_type, 'start_time': start_time, 'end_time': end_time})
        
    def get_conditional_order_history(self, market: str = None, side: str = None, type: str = None, order_type: str = None, start_time: float = None, end_time: float = None) -> List[dict]:
        return self._get(f'conditional_orders/history', {'market': market, 'side': side, 'type': type, 'orderType': order_type, 'start_time': start_time, 'end_time': end_time})

    def modify_order(
        self, existing_order_id: Optional[str] = None,
        existing_client_order_id: Optional[str] = None, price: Optional[float] = None,
        size: Optional[float] = None, client_order_id: Optional[str] = None,
    ) -> dict:
        assert (existing_order_id is None) ^ (existing_client_order_id is None), \
            'Must supply exactly one ID for the order to modify'
        assert (price is None) or (size is None), 'Must modify price or size of order'
        path = f'orders/{existing_order_id}/modify' if existing_order_id is not None else \
            f'orders/by_client_id/{existing_client_order_id}/modify'
        return self._post(path, {
            **({'size': size} if size is not None else {}),
            **({'price': price} if price is not None else {}),
            ** ({'clientId': client_order_id} if client_order_id is not None else {}),
        })

    def get_conditional_orders(self, market: str = None) -> List[dict]:
        return self._get(f'conditional_orders', {'market': market})

    def place_order(self, market: str, side: str, price: float, size: float, type: str = 'limit',
                    reduce_only: bool = False, ioc: bool = False, post_only: bool = False,
                    client_id: str = None) -> dict:
        return self._post('orders', {'market': market,
                                     'side': side,
                                     'price': price,
                                     'size': size,
                                     'type': type,
                                     'reduceOnly': reduce_only,
                                     'ioc': ioc,
                                     'postOnly': post_only,
                                     'clientId': client_id,
                                     })

    def place_conditional_order(
        self, market: str, side: str, size: float, type: str = 'stop',
        limit_price: float = None, reduce_only: bool = False, cancel: bool = True,
        trigger_price: float = None, trail_value: float = None
    ) -> dict:
        """
        To send a Stop Market order, set type='stop' and supply a trigger_price
        To send a Stop Limit order, also supply a limit_price
        To send a Take Profit Market order, set type='trailing_stop' and supply a trigger_price
        To send a Trailing Stop order, set type='trailing_stop' and supply a trail_value
        """
        assert type in ('stop', 'take_profit', 'trailing_stop')
        assert type not in ('stop', 'take_profit') or trigger_price is not None, \
            'Need trigger prices for stop losses and take profits'
        assert type not in ('trailing_stop',) or (trigger_price is None and trail_value is not None), \
            'Trailing stops need a trail value and cannot take a trigger price'

        return self._post('conditional_orders',
                          {'market': market, 'side': side, 'triggerPrice': trigger_price,
                           'size': size, 'reduceOnly': reduce_only, 'type': 'stop',
                           'cancelLimitOnTrigger': cancel, 'orderPrice': limit_price})

    def cancel_order(self, order_id: str) -> dict:
        return self._delete(f'orders/{order_id}')

    def cancel_orders(self, market_name: str = None, conditional_orders: bool = False,
                      limit_orders: bool = False) -> dict:
        return self._delete(f'orders', {'market': market_name,
                                        'conditionalOrdersOnly': conditional_orders,
                                        'limitOrdersOnly': limit_orders,
                                        })

    def get_fills(self) -> List[dict]:
        return self._get(f'fills')

    def get_balances(self) -> List[dict]:
        return self._get('wallet/balances')

    def get_deposit_address(self, ticker: str) -> dict:
        return self._get(f'wallet/deposit_address/{ticker}')

    def get_positions(self, show_avg_price: bool = False) -> List[dict]:
        return self._get('positions', {'showAvgPrice': show_avg_price})

    def get_position(self, name: str, show_avg_price: bool = False) -> dict:
        return next(filter(lambda x: x['future'] == name, self.get_positions(show_avg_price)), None)

    def get_all_trades(self, market: str, start_time: float = None, end_time: float = None) -> List:
        ids = set()
        limit = 100
        results = []
        while True:
            response = self._get(f'markets/{market}/trades', {
                'end_time': end_time,
                'start_time': start_time,
            })
            deduped_trades = [r for r in response if r['id'] not in ids]
            results.extend(deduped_trades)
            ids |= {r['id'] for r in deduped_trades}
            print(f'Adding {len(response)} trades with end time {end_time}')
            if len(response) == 0:
                break
            end_time = min(parse_datetime(t['time']) for t in response).timestamp()
            if len(response) < limit:
                break
        return results

ModuleNotFoundError: No module named 'ciso8601'